In [1]:
import sys 
sys.path.append("..") 
sys.path.append("../..") 

import ENV
import numpy as np
import pandas as pd
from DQN import Buffer
from DQN import DQN as DQN
import tensorflow as tf
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import os
import time

In [3]:
Train=True
init_train=False

tf.compat.v1.reset_default_graph()

data = pd.read_csv('../DATA/rdata.csv').iloc[:,1:]
index = np.load('../DATA/index.npy',allow_pickle=True).tolist()
action_table = np.load('../DATA/action_table1.npy',allow_pickle=True).tolist()

step = 24

agent_params={
    'state_dim':len(index['state']),
    'action_dim':len(action_table),

    'encoding_layer':[50,50],
    'value_layer':[50,50],
    'advantage_layer':[50,50],
    'test_num':100, # see pretrain_data

    'train_iterations':2,
    'training_step':100,
    'gamma':0.01,
    'epsilon':0.1,
    'ep_min':1e-50,
    'ep_decay':0.9,
    'learning_rate':0.001,

    'action_table':np.array(action_table),
}

model = DQN.DQN(agent_params)
if init_train:
    model.model.save_weights('./model/dqn.h5')    
    model.target_model.save_weights('./model/target_dqn.h5')    
model.load_model('./model/')

In [4]:
buffer = Buffer.Buffer(model.params['state_dim'], int(data.shape[0])*model.params['test_num'])

In [15]:
optdata = np.load('../DATA/pretrain_data/pretraindata0.npy',allow_pickle=True).tolist()
o=optdata['state'][1:-2]
a=optdata['action'][1:-1]
r=optdata['reward'][1:-2]
o_=optdata['state'][2:-1]
print(len(o),len(o[0]))

21 17


In [17]:
res = []
        
episode_return, episode_length = 0, 0
optdata = np.load('../DATA/pretrain_data/pretraindata0.npy',allow_pickle=True).tolist()
o=optdata['state'][1:-2]
a=optdata['action'][1:-1]
r=optdata['reward'][1:-2]
o_=optdata['state'][2:-1]
last_value=0
episode_return += np.sum(r)
episode_length += len(r)
res.append([o,a,r,o_,last_value,episode_return,episode_length])

In [5]:
def interact(timepoint,step,controlmodel):
    env = ENV.ENV()
    results = env.reset(timepoint,step)
    for s in range(step-1):
        observation = np.array(data.values[env.timepoint + 1 + step,env.title['inflow']].tolist()+results['state'][-1][-1]).reshape((1,-1))
        a = DQN.sample_action(observation,controlmodel,False)
        #_, a = PPO.sample_action(observation,ppomodel,False)
        action = action_table[a.numpy()[0]]
        results, done = env.step(action)
    
    return results

In [6]:
res = []
for timepoint in range(step,model.params['test_num']+step-3):
    episode_return, episode_length = 0, 0
    optdata = interact(timepoint,step,model)
    o=optdata['state'][1:-2]
    a=optdata['action'][1:-1]
    r=optdata['rewards'][1:-2]
    o_=optdata['state'][2:-1]
    last_value=0
    episode_return += np.sum(r)
    episode_length += len(r)
    res.append([o,a,r,o_,last_value,episode_return,episode_length])

In [8]:
np.array(optdata['state'][-1]).shape

(24, 9)

In [37]:
y = np.squeeze(np.array(optdata['state'][0]))
y.shape

(24, 9)

In [ ]:
for o,a,r,o_ in zip(res[i][0],res[i][1],res[i][2],res[i][3]):
    #buffer.store(o,a,r,o_) # For OPT data
    #For EFD data
    atem = 0
    for it in range(agent_params['action_table'].shape[0]):
        if (agent_params['action_table'][it] == np.array(a)).all():
            atem = it
    print(o)
    print(atem)
    print(r)
    print(o_)
    print('8888888888888888888888888888888888888888888888888888888888888888')
    buffer.store(o,atem,r,o_)

In [9]:
env = ENV.ENV()

In [11]:
len(env.title['state']),len(env.title['inflow'])

(9, 8)

In [ ]:
Train=True
init_train=False

tf.compat.v1.reset_default_graph()

data = pd.read_csv('../../DATA/rdata.csv').iloc[:,1:]
index = np.load('../../DATA/index.npy',allow_pickle=True).tolist()
action_table = np.load('../../DATA/action_table1.npy',allow_pickle=True).tolist()

step = 24

agent_params={
    'state_dim':len(index['state']),
    'action_dim':len(action_table),

    'encoding_layer':[50,50],
    'value_layer':[50,50],
    'advantage_layer':[50,50],
    'test_num':100, # see pretrain_data

    'train_iterations':2,
    'training_step':100,
    'gamma':0.01,
    'epsilon':0.1,
    'ep_min':1e-50,
    'ep_decay':0.9,
    'learning_rate':0.001,

    'action_table':np.array(action_table),
}

model = DQN.DQN(agent_params)
if init_train:
    model.model.save_weights('./model/dqn.h5')    
    model.target_model.save_weights('./model/target_dqn.h5')    
model.load_model('./model/')
print('model done')

model done


In [ ]:
def interact(timepoint,step,controlmodel):
    env = ENV.ENV()
    results = env.reset(timepoint,step)
    print(timepoint,step,timepoint+step+step+1)
    for s in range(step):
        observation = np.array(data.values[env.timepoint + 1 + step,env.title['inflow']].tolist()+results['state'][-1][-1]).reshape((1,-1))
        a = DQN.sample_action(observation,controlmodel,False)
        #_, a = PPO.sample_action(observation,ppomodel,False)
        action = action_table[a.numpy()[0]]
        results, done = env.step(action)
    
    return results

In [ ]:
# main training process   
history = {'episode': [], 'Batch_reward': [], 'Episode_reward': [], 'Loss': []}

# Iterate over the number of epochs
for epoch in range(model.params['training_step']):
    # Initialize the sum of the returns, lengths and number of episodes for each epoch
    sum_return = 0
    sum_length = 0
    num_episodes = 0
    
    # Initialize the buffer
    buffer = Buffer.Buffer(model.params['state_dim'], int(data.shape[0])*model.params['test_num'])
    
    # Iterate over the steps of each epoch
    res = []
    for timepoint in range(step,step+model.params['test_num']):
        episode_return, episode_length = 0, 0
        optdata = interact(timepoint,step,model)
        o=optdata['state'][1:-2][0]
        a=optdata['action'][1:-1]
        r=optdata['rewards'][1:-2]
        o_=optdata['state'][2:-1][0]
        last_value=0
        episode_return += np.sum(r)
        episode_length += len(r)
        res.append([o,a,r,o_,last_value,episode_return,episode_length])

    for i in range(model.params['test_num']):
        #s, a, r, vt, lo, lastvalue in buffer
        for o,a,r,o_ in zip(res[i][0],res[i][1],res[i][2],res[i][3]):
            #buffer.store(o,a,r,o_) # For OPT data
            #For EFD data
            atem = 0
            for it in range(agent_params['action_table'].shape[0]):
                if (agent_params['action_table'][it] == np.array(a)).all():
                    atem = it
            buffer.store(o,atem,r,o_)
        buffer.finish_trajectory(res[i][4])
        sum_return += res[i][5]
        sum_length += res[i][6]
        num_episodes += 1
    
    # Get values from the buffer
    (
        observation_buffer,
        action_buffer,
        observation_next_buffer,
        reward_buffer,
        advantage_buffer,
    ) = buffer.get()

    # Update the policy and implement early stopping using KL divergence
    for _ in range(model.params['train_iterations']):
        l=DQN.train_value(observation_buffer, action_buffer, reward_buffer, observation_next_buffer, model)
        history['Loss'].append(l)

FileNotFoundError: [Errno 2] No such file or directory: '../Step0_DNN/DATA/rdata.csv'